In [1]:
import pandas as pd
import polars as pl
import altair as alt
import vegafusion as vf
vf.enable()
vf.enable_widget()

vegafusion.enable_widget()

In [2]:
flights = (
    pl.read_parquet("https://vegafusion-datasets.s3.amazonaws.com/vega/flights_1m.parquet")
)

In [3]:
flights

index,date,delay,distance,origin,destination
i64,datetime[ns],i64,i64,str,str
0,2001-02-14 15:12:00,8,342,"""SJC""","""SNA"""
1,2001-01-22 09:50:00,-10,601,"""PHX""","""RNO"""
2,2001-01-17 10:59:00,4,181,"""DAL""","""OKC"""
3,2001-01-14 16:20:00,4,630,"""PDX""","""SLC"""
4,2001-03-14 10:50:00,-3,223,"""BUR""","""LAS"""
…,…,…,…,…,…
9995,2001-03-04 11:48:00,60,1073,"""MCO""","""PVD"""
9996,2001-03-26 21:20:00,0,308,"""SJC""","""LAX"""
9997,2001-03-22 09:40:00,-6,248,"""SAT""","""DAL"""


In [4]:
delay_hist = flights.plot.bar(
    alt.X("delay", bin=alt.Bin(maxbins=30)),
    alt.Y("count()")
)

delay_hist

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…

In [5]:
vf.transformed_data(delay_hist)

bin_maxbins_30_delay,bin_maxbins_30_delay_end,__count
f64,f64,i64
0.0,20.0,392700
-20.0,0.0,419400
20.0,40.0,92700
140.0,160.0,2000
40.0,60.0,38400
…,…,…
260.0,280.0,100
200.0,220.0,300
360.0,380.0,100


In [5]:
brush = alt.selection_interval(encodings=["x"])

base = alt.Chart().mark_bar().encode(
    x = alt.X(alt.repeat('column')).bin(maxbins=20),
    y = 'count()'
).properties(
    width=160,
    height=130
)

background = base.encode(
    color=alt.value('#ddd')
).add_params(brush)

highlight = base.transform_filter(brush)

chart = alt.layer(
    background,
    highlight,
    data=flights
).transform_calculate(
    "time",
    "hours(datum.date)"
).repeat(column=["distance", "delay", "time"])

In [6]:
chart

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 300,\n      "continuousHeight…